In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# double check your tf version
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# get sampling data set
fashion_mnist = keras.datasets.mnist
(trainx, trainy), (testx, testy) = fashion_mnist.load_data()
trainx = (trainx.astype(np.float32) - 127.5) / 127.5

trainx = trainx[trainy == 0]
testx = testx[testy == 0]

trainx = trainx.reshape(trainx.shape[0],28,28,1)
testx = testx.reshape(testx.shape[0],28,28,1)

print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

2.0.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3933932169061679741
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4937233203
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16991730595437065990
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
(5923, 28, 28, 1)
(60000,)
(980, 28, 28, 1)
(10000,)


In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://gluon.mxnet.io/_images/dcgan.png")

In [3]:
class myGAN():
    def __init__(self, width=28, height=28, channels=1):
        self.width = width
        self.height = height
        self.channels = channels
        self.shape = (self.width, self.height, self.channels)
        self.optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5, decay=8e-8)
       
        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])
        
        self.GAN = self.__gan()
        self.GAN.compile(loss='binary_crossentropy', optimizer=self.optimizer)        
      
    # use vector as input.
    def __generator(self):
        model = tf.keras.Sequential(
            [
                # GAN generator need to generate a batch of Samples, so here is 256 images with small size 7*7
                # flatten layer: 7*7
                tf.keras.layers.Dense(256*7*7, input_shape=(100,)),                
                tf.keras.layers.BatchNormalization(),  # 使用 BatchNormalization 優化
                tf.keras.layers.LeakyReLU(),   # 使用 LeakyReLU 激活函數
                
                # reshape
                tf.keras.layers.Reshape((7, 7, 256)),
                ### model.output_shape == (None, 7,7,256)
                
                # layer 2
                # self.G.add(UpSampling2D())
                tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
                 ### model.output_shape == (None, 7,7,128)
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.LeakyReLU(),
                                
                # layer 3
                # self.G.add(UpSampling2D())
                tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
                 ### model.output_shape == (None, 14,14,64)
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.LeakyReLU(),
                
                # layer 4
                tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
                ### model.output_shape == (None, 28,28,1)        
            ]
        )
        model.summary()
        return model
    
    def __discriminator(self):
        model = tf.keras.Sequential( 
            [
                # input 28*28, 64 filters and each is 5 * 5 , stride means step size for filter              
                tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',input_shape=[28, 28, 1]),
                tf.keras.layers.LeakyReLU(),
                tf.keras.layers.Dropout(0.3),
                
                # input 14*14
                tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
                tf.keras.layers.LeakyReLU(),
                tf.keras.layers.Dropout(0.3),                
                
                tf.keras.layers.Conv2D(64*4, (5, 5), strides=(2, 2), padding='same'),
                tf.keras.layers.LeakyReLU(),
                tf.keras.layers.Dropout(0.3),
                
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(1)
            ]
        )
        model.summary()
        return model
    
    def __gan(self):
        self.D.trainable = False
        self.G.trainable = True

        model = tf.keras.Sequential(
            [
                self.G,
                self.D
            ]
        )

        return model
    
    def __generator_loss(fake_output):
        cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        
        return cross_entropy(tf.ones_like(fake_output), fake_output)
    
    def __discriminator_loss(real_output, fake_output):
        # This method returns a helper function to compute cross entropy loss
        cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        
        return total_loss
    
    # 1. fix Generator's parameter and generate randomly sample vector output.
    # 2. get real sample from DB
    # 3. Train Discriminator with real sample + bad sample
    # 4. fix Discriminator's parameter and train Generator.

    def train_test(self, X_train, epochs=50, batch = 32, save_interval = 100):
        for epoch in range(epochs):
            index = np.random.randint(0, X_train.shape[0], size=batch)
            images_train = X_train[index]
            
            noise = np.random.normal(0.0,1.0,(batch, 100))
            images_fake = self.G(noise)
            
            x = np.concatenate((images_train, images_fake))
            y = np.concatenate((np.ones((batch, 1)), np.zeros((batch, 1))))
            
            d_loss = self.D.train_on_batch(x,y)

            #y[batch:,:] = 1
            noise = np.random.normal(0, 1.0, size=[2*batch, 100])
            y_mislabeld = np.ones((2*batch, 1))
            
            a_loss = self.GAN.train_on_batch(noise, y_mislabeld)
            
            if epoch % 100 == 0:
                print('epoch: ',epoch,', [Discriminator :: d_loss: ',d_loss,'], [ Generator :: loss: ',a_loss,']')           
                self.plot_images(True,16,epoch)
         
    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        filename = "./Result/tf20_dcgan/mnist_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height, self.width]).astype('float32')
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [4]:
tf.keras.backend.set_floatx('float32')
#gpu_options = tf.GPUOptions(allow_growth=True)

go = myGAN()
print(len(trainx))
go.train_test(trainx, 5000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 256)         8

W1020 17:03:39.566539  8796 base_layer.py:1814] Layer dense_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



5923
epoch:  0 , [Discriminator :: d_loss:  [3.1855884, 0.5] ], [ Generator :: loss:  3.099935 ]
epoch:  100 , [Discriminator :: d_loss:  [0.0, 1.0] ], [ Generator :: loss:  0.0 ]
epoch:  200 , [Discriminator :: d_loss:  [0.0018986292, 1.0] ], [ Generator :: loss:  0.0 ]
epoch:  300 , [Discriminator :: d_loss:  [0.02529014, 0.984375] ], [ Generator :: loss:  0.00085635914 ]
epoch:  400 , [Discriminator :: d_loss:  [7.7124743, 0.5] ], [ Generator :: loss:  15.424949 ]
epoch:  500 , [Discriminator :: d_loss:  [0.0058436114, 1.0] ], [ Generator :: loss:  0.1024688 ]
epoch:  600 , [Discriminator :: d_loss:  [0.19720873, 0.921875] ], [ Generator :: loss:  4.969721 ]
epoch:  700 , [Discriminator :: d_loss:  [1.5223641, 0.8125] ], [ Generator :: loss:  5.142622 ]
epoch:  800 , [Discriminator :: d_loss:  [0.40199584, 0.84375] ], [ Generator :: loss:  3.205265 ]
epoch:  900 , [Discriminator :: d_loss:  [0.2425319, 0.96875] ], [ Generator :: loss:  3.7721841 ]
epoch:  1000 , [Discriminator :: d_